In [1]:
# pip install workspace

In [2]:
# pip install dataset

In [3]:
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
ws = Workspace.from_config()

In [5]:
data = pd.read_csv('diabetes2.csv')
data.head()

,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1142956,1,78,41,33,311,50.796392,0.420804,24,0
1,1823377,0,116,92,16,184,18.603630,0.131156,22,0
2,1916381,8,171,42,29,160,35.482247,0.082671,22,1
3,1247480,3,108,63,45,297,49.375169,0.100979,46,1
4,1516947,8,153,99,15,41,35.062139,0.116191,22,1


In [6]:
data.columns

Index(['PatientID', 'Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure',
       'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age',
       'Diabetic'],
      dtype='object')

In [25]:
x_train,x_test = train_test_split(data, test_size = 0.2)

In [8]:
automl_settings={
    "iteration_timeout_minutes":2,
    "experiment_timeout_minutes":15,
    "enable_early_stopping":True,
    "primary_metric":'AUC_weighted',
    "featurization": 'auto',
    "n_cross_validations":5,
    
}

In [9]:
automl_config = AutoMLConfig(task='classification',
    debug_logs='aotoml_errors.log',
    training_data=x_train,
    label_column_name= "Diabetic",
    **automl_settings)

In [10]:
experiment = Experiment(ws, 'diabetesExpe')

In [11]:
# pip install -r /anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/validated_linux_requirements.txt

In [12]:
# pip install markupsafe==2.0.1

In [13]:
run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
diabetesExpe,AutoML_8e5965c5-3e6e-44d2-9f65-2806c235f1da,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


2024/06/12 11:29:02 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality featu

********************************************************************************************
Current status: BestRunExplainModel. Best run model explanations started
Current status: ModelExplanationDataSetSetup. Model explanations data setup completed
Current status: PickSurrogateModel. Choosing LightGBM as the surrogate model for explanations
Current status: EngineeredFeatureExplanations. Computation of engineered features started


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


In [14]:
best_run, model = run.get_output()

In [15]:
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [16]:
model_name = best_run.properties["model_name"]
registered_name = run.register_model(model_name=model_name, description="AutoML Diabetes",tags=None)

In [22]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment
from azureml.core.model import Model

In [23]:
best_run.download_file("outputs/scoring_file_v_1_0_0.py","inference/score.py")

In [24]:
from azureml.automl.core.shared import constants

best_run.download_file(constants.CONDA_ENV_FILE_PATH, "myenv.yml")
env = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

inference_config = InferenceConfig(entry_script="inference/score.py",environment=env)
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, description="Diabetes Classification")
service = Model.deploy(ws, "automl-diabetes", [registered_name], inference_config,aciconfig)
 
service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-06-12 12:01:51+00:00 Creating Container Registry if not exists..
2024-06-12 12:11:52+00:00 Registering the environment..
2024-06-12 12:11:54+00:00 Building image..
2024-06-12 12:31:41+00:00 Generating deployment configuration.
2024-06-12 12:31:41+00:00 Submitting deployment to compute..
2024-06-12 12:31:48+00:00 Checking the status of deployment automl-diabetes..
2024-06-12 12:34:24+00:00 Checking the status of inference endpoint automl-diabetes.
Succeeded
ACI service creation operation finished, operation "Succeeded"
